In [1]:
from LDistanceModel import ldistance

# Example input strings
str1 = "kitten"
str2 = "sitting"

# Function call to calculate Levenshtein distance
distance = ldistance(str1, str2)

# Print the result
print("Levenshtein Distance:", distance)
import json
print("Json:", json.dumps(distance))

data = distance

Levenshtein Distance: [('Substitute', 'k', 's'), ('Nothing', 'i'), ('Nothing', 't'), ('Nothing', 't'), ('Substitute', 'e', 'i'), ('Nothing', 'n'), ('Insert', 'g')]
Json: [["Substitute", "k", "s"], ["Nothing", "i"], ["Nothing", "t"], ["Nothing", "t"], ["Substitute", "e", "i"], ["Nothing", "n"], ["Insert", "g"]]


In [2]:
str1 = "kitten"
str2 = "sitting"
unique_letters = set()
unique_letters.update(str1, str2)

unique_letters

{'e', 'g', 'i', 'k', 'n', 's', 't'}

In [3]:
embedding_dim = 4

from LDistanceModel import SetEmbedding

embedding = SetEmbedding(set_of_things=unique_letters, embedding_dim=embedding_dim)

embedding('g')

tensor([[-0.0201,  1.0752,  0.0275, -1.1097]], grad_fn=<EmbeddingBackward0>)

In [4]:
data

[('Substitute', 'k', 's'),
 ('Nothing', 'i'),
 ('Nothing', 't'),
 ('Nothing', 't'),
 ('Substitute', 'e', 'i'),
 ('Nothing', 'n'),
 ('Insert', 'g')]

In [5]:
from LDistanceModel import operation_encoder

operation_encoder(data, embedding)

tensor([[ 0.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.5000, -1.7438, -0.3490, -3.5093,  1.0246],
        [ 0.0000,  0.0000,  2.1463, -1.1699, -0.2154,  0.4448],
        [ 0.0000,  0.0000, -1.7527,  0.7580,  0.4855,  0.2966],
        [ 0.0000,  0.0000, -1.7527,  0.7580,  0.4855,  0.2966],
        [ 0.5000,  0.0000,  1.4606, -1.2639, -1.1067, -0.3888],
        [ 0.0000,  0.5000,  2.1463, -1.1699, -0.2154,  0.4448],
        [ 0.0000,  0.0000, -0.5340, -1.7844,  1.1286, -0.7992],
        [ 0.0000,  1.0000, -0.0201,  1.0752,  0.0275, -1.1097]],
       grad_fn=<CopySlices>)

In [6]:
string_pairs = [
    ("hello", "hallo"),
    ("cat", "bat"),
    ("python", "java"),
    ("same", "same"),
    ("a", "abc")
]

all_characters_in_pairs = set()
for pair in string_pairs:
    for word in pair:
        all_characters_in_pairs.update(word)

string_pair_embedder = SetEmbedding(all_characters_in_pairs, 10)

from LDistanceModel import create_batch_with_masks
create_batch_with_masks(string_pairs, string_pair_embedder)

(tensor([[[ 0.0000,  0.0000, -2.0093,  2.5259, -0.4276,  1.8567, -2.1050,
            0.2206,  1.1253,  1.1538,  1.1189,  0.7314],
          [ 0.5000,  0.0000,  0.6047, -1.2549,  0.3655,  0.6385, -2.5045,
            0.8770,  0.5173, -0.6846, -0.4493,  0.9730],
          [ 0.0000,  0.5000,  0.5111, -1.1539, -0.2442, -1.0754, -0.6968,
            0.2592,  1.4359,  0.6875, -0.7560,  0.3229],
          [ 0.0000,  0.0000,  0.8474, -0.5229,  0.2423, -1.1077, -1.0637,
            2.8645,  0.4781, -0.2449, -0.4695, -0.1594],
          [ 0.0000,  0.0000,  0.8474, -0.5229,  0.2423, -1.1077, -1.0637,
            2.8645,  0.4781, -0.2449, -0.4695, -0.1594],
          [ 0.0000,  0.0000, -0.2216,  0.3599, -0.0070,  1.4073, -0.4181,
           -1.1262,  0.2773, -0.8467,  0.8706, -0.2134],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
            0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         

In [12]:
import torch

bruh = torch.tensor([1,2,3])
yeet = torch.tensor([1,2,3])

torch.stack((bruh, yeet))[:, 2]

[[1,2,3],[1,2,3]][:, 2]

<>:8: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:8: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\lsm0147\AppData\Local\Temp\ipykernel_36828\2517620060.py:8: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [[1,2,3],[1,2,3]][:, 2]


TypeError: list indices must be integers or slices, not tuple

In [7]:
import graphviz

# Create a graphviz Digraph object
dot = graphviz.Digraph()

# Function to add nodes and edges to the graph in reverse order, skipping nodes with operation 'Nothing'
def add_nodes_edges_reverse_skip_nothing(data, parent=None):
    if data['operation'][0] == 'Nothing':
        if data['history']:
            add_nodes_edges_reverse_skip_nothing(data['history'], parent)
        return
    
    node_id = str(id(data))
    label = f"Distance: {data['distance']}\nOperation: {data['operation']}"
    dot.node(node_id, label)
    
    if parent:
        dot.edge(node_id, parent)
    
    if data['history']:
        add_nodes_edges_reverse_skip_nothing(data['history'], node_id)

# Add nodes and edges starting from the root
add_nodes_edges_reverse_skip_nothing(data)

# Save the graph to a file
dot.save('graph_reverse_skip_nothing_new.dot')

# Render the graph to a PNG image file
dot.render('graph_reverse_skip_nothing_new', format='png')

print("Graph has been successfully created and saved as graph_reverse_skip_nothing_new.png.")

TypeError: list indices must be integers or slices, not str